In [2]:
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator( QComponent ):
    """ A double chip resonator devic "vrunda_design .ipynb"e "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            trans_orientation = '90',
                            trans_qubit_rotation = '0',
                            trans_rect_height = '0.23mm',
                            trans_rect_width = '0.5mm',
                            trans_rect_pos_x = '7.35mm',
                            trans_rect_pos_y = '0mm',
                            trans_pad_width = '0.45mm',
                            trans_pad_height = '0.12mm',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_width/2
        pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_height,pos_x, pos_y)
        r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_height, pos_x, pos_y)
        transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_width
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  

        #################################### phi ##################################################################################
        diff_rect_pad_width = (p.trans_rect_width - p.trans_pad_width)/2
        R1 = r_transmon - p.trans_pad_width/2  - diff_rect_pad_width/2
        a = diff_rect_pad_width/2 + p.trans_pad_width/2
        # phi = math.acos( (R1**2 + r_transmon**2 - a**2) / 2 *r_transmon*R1) + (p.trans_orientation * 180)/math.pi
        
        #################################### josephson_junction ######################################################################
        
        r_1 = R1 - diff_rect_pad_width/2
        r_2 = R1 + diff_rect_pad_width/2
        # R1_x = R1 * math.cos( p.trans_orientation * math.pi / 180 ) 
        # R1_y = R1 * math.sin( p.trans_orientation * math.pi /180 )
        r_1_x = r_1 * math.cos(p.trans_orientation * math.pi / 180 ) 
        r_1_y = r_1 * math.sin(p.trans_orientation  * math.pi / 180 ) 
        r_2_x = r_2 * math.cos(p.trans_orientation  * math.pi / 180 ) 
        r_2_y = r_2 * math.sin(p.trans_orientation  * math.pi / 180 ) 

        joseph = draw.LineString([(r_2_x, r_2_y),(r_1_x, r_1_y)])

        
        final_design_1 = draw.subtract( outer_circle, r1)
        final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( final_design_2, inner_circle)
        objects = [final_design_3,joseph]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3,joseph] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)
        self.add_qgeometry('junction', {'joseph': joseph},
                           subtract=True,
                           width= diff_rect_pad_width,
                           layer=p.layer,
                           chip=p.chip)
        
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator_u( QComponent ):
    """ A double chip resonator device "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            # trans_orientation = '90',
                            # trans_qubit_rotation = '0',
                            trans_rect_len = '0.5mm',
                            # trans_rect_width = '0.23mm',
                            # trans_rect_pos_x = '7.35mm',
                            # trans_rect_pos_y = '0mm',
                            # trans_pad_width = '0.12mm',
                            # trans_pad_len = '0.45mm',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        # r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_len/2
        # pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        # pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        # r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_len,pos_x, pos_y)
        # r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_len, pos_x, pos_y)
        # transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        # inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_len
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  
        # final_design_1 = draw.subtract( outer_circle, r1)
        # final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( outer_circle, inner_circle)
        objects = [final_design_3]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)


In [3]:
from qiskit_metal import designs, MetalGUI
design  = designs.DesignFlipChip()
gui = MetalGUI( design ) 
gui.overwrite_enabled = True


design.overwrite_enabled = True
q1 =  Whisper_gallery_resonator( design, "Q1", options = Dict(inner_cir_rotation = '0', trans_orientation = '0',chip = 'Q_chip') )
gui.rebuild()
gui.autoscale()

04:58PM 15s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
04:58PM 15s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`


In [4]:
gui.overwrite_enabled = True 
q2  =  Whisper_gallery_resonator_u( design, 'Q2', options = Dict( inner_cir_rotation = '0', trans_orientation = '0',chip = 'C_chip'))
gui.rebuild()
gui.autoscale()

04:58PM 16s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`
04:58PM 16s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
04:58PM 16s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`


In [9]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")
from qiskit_metal.analyses.quantization import EPRanalysis
import pyEPR as epr
eig_res = EPRanalysis( design, 'hfss')
hfss = eig_res.sim.renderer

In [10]:
sim_p = c1.sim.setup

In [11]:
sim_p.name = 'QubitTune'
sim_p.max_passes = 15
sim_p.min_converged_passes = 2
sim_p.percent_error = 0.1

c1.sim.setup

{'name': 'QubitTune',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 15,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.1,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [15]:
# hfss.start()
q3d = c1.sim.renderer

In [16]:
# c1.sim.run(name = 'FLIIP_CHIP_LOManalysis')
q3d.start()

INFO 05:03PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:03PM [load_ansys_project]: 	Opened Ansys App
INFO 05:03PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 05:03PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Documents/Ansoft/
	Project:   Project37
INFO 05:03PM [connect_design]: No active design found (or error getting active design).
INFO 05:03PM [connect]: 	 Connected to project "Project37". No design detected


True

In [17]:
q3d.activate_ansys_design("flip_chip", 'capacitive')

05:04PM 52s WARNING [activate_ansys_design]: The design_name=flip_chip was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 05:05PM [connect_design]: 	Opened active design
	Design:    flip_chip [Solution type: Q3D]
WARNING 05:05PM [connect_setup]: 	No design setup detected.
WARNING 05:05PM [connect_setup]: 	Creating Q3D default setup.
INFO 05:05PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


In [18]:
q3d.render_design()

In [20]:
q3d.analyze_setup()

TypeError: QQ3DRenderer.analyze_setup() missing 1 required positional argument: 'setup_name'